In [47]:
import pandas as pd
import numpy as np
import sklearn as sk
import string
import itertools
from collections import Counter
from gensim.models import Word2Vec, KeyedVectors
from Levenshtein import distance as lev
from ast import literal_eval

In [2]:
#def extractData():
#noteEvents=pd.read_csv("mimic-iii-clinical-database-1.4/NOTEEVENTS.csv")
noteEvents = pd.read_csv("NOTEEVENTS.csv")
noteEvents=noteEvents[["HADM_ID","TEXT","DESCRIPTION"]]
#diagnosis = pd.read_csv("mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv")
diagnosis = pd.read_csv("DIAGNOSES_ICD.csv")
diagnosis = diagnosis[["HADM_ID","SUBJECT_ID","ICD9_CODE"]]

C:\Users\xin\AppData\Local\Temp/ipykernel_10152/3783482439.py:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  noteEvents = pd.read_csv("NOTEEVENTS.csv")


In [3]:
def convert_icd9(x):
    converted_code = []
    for code in x:
        icd9_str = str(code)
        if icd9_str.startswith("E"):
            converted = icd9_str[:4]
        else:
            converted = icd9_str[:3]
        converted_code.append(converted)
    return converted_code

def checkdiagnosis(x):
    for code in x:
        if str(code)[0:3]=="250":
            return True
    return False

In [4]:
#Group Diagnosis into 1 list per admission
diagnosisGrouped = diagnosis.groupby(["SUBJECT_ID","HADM_ID"])["ICD9_CODE"].apply(list).reset_index()
diagnosisGrouped.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,2,163353,"[V3001, V053, V290]"
1,3,145834,"[0389, 78559, 5849, 4275, 41071, 4280, 6826, 4..."
2,4,185777,"[042, 1363, 7994, 2763, 7907, 5715, 04111, V09..."
3,5,178980,"[V3000, V053, V290]"
4,6,107064,"[40391, 4440, 9972, 2766, 2767, 2859, 2753, V1..."


In [6]:
admissionswithDiabetes = diagnosisGrouped[diagnosisGrouped["ICD9_CODE"].apply(lambda x: checkdiagnosis(x))]
admissionswithDiabetes["ICD9_CODE_primary"] = admissionswithDiabetes["ICD9_CODE"].apply(lambda x: convert_icd9(x))
admissionswithDiabetes.head()

C:\Users\xin\AppData\Local\Temp/ipykernel_10152/1912625630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  admissionswithDiabetes["ICD9_CODE_primary"] = admissionswithDiabetes["ICD9_CODE"].apply(lambda x: convert_icd9(x))


,SUBJECT_ID,HADM_ID,ICD9_CODE,ICD9_CODE_primary
11,13,143045,"[41401, 4111, 25000, 4019, 2720]","[414, 411, 250, 401, 272]"
15,18,188822,"[25080, 78039, 29633, V5867, E9323, V5869, 478...","[250, 780, 296, V58, E932, V58, 478, 780, 783,..."
17,20,157681,"[41401, 4111, 25000, 2724, 4019]","[414, 411, 250, 272, 401]"
18,21,109451,"[41071, 78551, 5781, 5849, 40391, 4280, 4592, ...","[410, 785, 578, 584, 403, 428, 459, 507, 427, ..."
19,21,111970,"[0388, 78552, 40391, 42731, 70709, 5119, 6823,...","[038, 785, 403, 427, 707, 511, 682, 998, 008, ..."


In [7]:
combineddf=noteEvents.merge(admissionswithDiabetes, on="HADM_ID")
print(combineddf.shape)
combineddf.head()

(406203, 6)


,HADM_ID,TEXT,DESCRIPTION,SUBJECT_ID,ICD9_CODE,ICD9_CODE_primary
0,121936.0,Admission Date: [**2125-2-9**] D...,Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
1,121936.0,PATIENT/TEST INFORMATION:\nIndication: Aortic ...,Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
2,121936.0,PATIENT/TEST INFORMATION:\nIndication: Aortic...,Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
3,121936.0,Sinus rhythm. Frequent atrial premature beats...,Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
4,121936.0,Rhythm is most likely sinus rhythm with freque...,Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."


In [ ]:
#filtereddf.to_csv("PreFilteredData.csv")
#filtereddf = pd.read_csv("PreFilteredData.csv")

In [8]:
processeddf=combineddf.copy()

In [9]:
spacestring=" "*len(string.punctuation.replace("'",""))
dstring="d"*len(string.digits)

In [10]:
processeddf["TEXT"]=processeddf["TEXT"].apply(lambda x: x.translate(str.maketrans(string.punctuation.replace("'",""),spacestring)))
processeddf["TEXT"] = processeddf["TEXT"].apply(lambda x: x.translate(str.maketrans(string.digits,dstring)))
processeddf["TEXT"] = processeddf["TEXT"].apply(lambda x: x.lower())
processeddf["TEXT"] = processeddf["TEXT"].apply(lambda x: x.split())

In [ ]:
#processeddf.to_csv("Text_Cleaned_Data.csv")
#processeddf = pd.read_csv("Text_Cleaned_Data.csv")
#processeddf["TEXT"] = processeddf["TEXT"].apply(literal_eval)

In [11]:
wordslist=processeddf["TEXT"].tolist()
flatlist=list(itertools.chain(*wordslist))
wordset=set(flatlist)
wordDict=Counter(flatlist)
wordDict5orMore= dict(filter(lambda x: x[1] >=5, wordDict.items())) 
wordDictLessThan5 = dict(filter(lambda x: x[1] <5, wordDict.items()))

In [12]:
print(len(wordDict5orMore)) #53229
print(len(wordDictLessThan5)) #109486

53229
109486


In [14]:
# Get get the closest lev-distance word
wordListLessThan5 = list(wordDictLessThan5.keys())
wordList5orMore = list(wordDict5orMore.keys())

mapped_dict = dict()

for w in range(len(wordListLessThan5)):
    lev_dist = []
    for i in range(len(wordList5orMore)):
        lev_dist.append(lev(wordListLessThan5[w],wordList5orMore[i]))
    #print(np.argmin(lev_dist))
    mapped_dict[wordListLessThan5[w]] = wordList5orMore[np.argmin(lev_dist)]
    
print(len(mapped_dict))
mapped_dict

109486


{'accesorry': 'accesory',
 'inferolaterl': 'inferolateral',
 'dheart': 'heart',
 'unpon': 'upon',
 'family\x13': 'family',
 'plebitis': 'phlebitis',
 'concsiousness': 'consiousness',
 'decipherable': 'indecipherable',
 'skd': 'sd',
 'dicloxcilllin': 'dicloxacillin',
 'distontinued': 'discontinued',
 'regment': 'segment',
 'quinepate': 'urinate',
 'vassopressor': 'vasopressor',
 'cadveric': 'cadaveric',
 'prophlaxsis': 'prophlaxis',
 'depresses': 'depressed',
 'imcompletely': 'incompletely',
 'ammiodarone': 'amiodarone',
 'dobuatimine': 'dobuatamine',
 'upholstery': 'holter',
 'intravenouses': 'intravenous',
 'notred': 'noted',
 'epigasrric': 'epigastric',
 'nadol': 'hadol',
 'nadadol': 'nadalol',
 'gluribide': 'glipizide',
 'lipotor': 'lipitor',
 'atenol': 'atenlol',
 'golytlely': 'golytely',
 'eftddd': 'tddd',
 'discolouration': 'discoloration',
 'balateral': 'bilateral',
 'gorlin': 'groin',
 'propecia': 'alopecia',
 'intervak': 'interval',
 'antegerade': 'antegrade',
 'undress': 'add

In [15]:
# filter the report based on length
limited_processeddf = processeddf[processeddf['TEXT'].apply(lambda x: len(x)) > 9]
limited_processeddf = limited_processeddf[limited_processeddf['TEXT'].apply(lambda x: len(x)) < 2200]
print(limited_processeddf.shape)
limited_processeddf.head()

(399623, 6)


,HADM_ID,TEXT,DESCRIPTION,SUBJECT_ID,ICD9_CODE,ICD9_CODE_primary
1,121936.0,"[patient, test, information, indication, aorti...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
2,121936.0,"[patient, test, information, indication, aorti...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
3,121936.0,"[sinus, rhythm, frequent, atrial, premature, b...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
4,121936.0,"[rhythm, is, most, likely, sinus, rhythm, with...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."
5,121936.0,"[atrial, fibrillation, intraventricular, condu...",Report,28063,"[42843, 41071, 5990, 4275, 5849, 5070, 4280, 2...","[428, 410, 599, 427, 584, 507, 428, 272, 401, ..."


In [19]:
def map_misspelled(x):
    new_l = [mapped_dict.get(item,item) for item in x]
    return new_l

In [31]:
# map misspelled words to correct words
limited_processeddf["TEXT"] = limited_processeddf["TEXT"].apply(lambda x: map_misspelled(x))

In [35]:
# check number of regular and rolled up code
icd9_list=limited_processeddf["ICD9_CODE_primary"].tolist()
icd9_flatlist=list(itertools.chain(*icd9_list))
print("Num. icd9 primary code",len(icd9_flatlist))
icd9_wordset=set(icd9_flatlist)
icd9_wordDict=Counter(icd9_flatlist)
print("Num. unique icd9 primary code",len(icd9_wordDict))
icd9_re_list=limited_processeddf["ICD9_CODE"].tolist()
icd9_re_flatlist=list(itertools.chain(*icd9_re_list))
print("Num. icd9 regular code",len(icd9_re_flatlist))
icd9_re_wordset=set(icd9_re_flatlist)
icd9_re_wordDict=Counter(icd9_re_flatlist)
print("Num. unique icd9 regular code",len(icd9_re_wordDict))

Num. icd9 primary code 6957549
Num. unique icd9 primary code 850
Num. icd9 regular code 6957549
Num. unique icd9 regular code 4097


In [34]:
limited_processeddf.to_csv("Tokenized.csv")

In [46]:
tokens= limited_processeddf["TEXT"].tolist()
tokens[:5]

399623


[['patient',
  'test',
  'information',
  'indication',
  'aortic',
  'valve',
  'disease',
  'left',
  'ventricular',
  'function',
  'height',
  'in',
  'dd',
  'weight',
  'lb',
  'ddd',
  'bsa',
  'md',
  'd',
  'dd',
  'md',
  'bp',
  'mm',
  'hg',
  'ddd',
  'dd',
  'hr',
  'bpm',
  'dd',
  'status',
  'inpatient',
  'date',
  'time',
  'dddd',
  'd',
  'dd',
  'at',
  'dd',
  'dd',
  'test',
  'portable',
  'tte',
  'complete',
  'doppler',
  'full',
  'doppler',
  'and',
  'color',
  'doppler',
  'contrast',
  'none',
  'technical',
  'quality',
  'adequate',
  'interpretation',
  'findings',
  'this',
  'study',
  'was',
  'compared',
  'to',
  'the',
  'report',
  'of',
  'the',
  'prior',
  'study',
  'images',
  'not',
  'available',
  'of',
  'dddd',
  'd',
  'd',
  'left',
  'atrium',
  'mild',
  'la',
  'enlargement',
  'right',
  'atrium',
  'interatrial',
  'septum',
  'mildly',
  'dilated',
  'ra',
  'left',
  'ventricle',
  'mild',
  'symmetric',
  'lvh',
  'with',
 

In [53]:
print("Num. total tokens",sum(map(len, tokens)))

Num. total tokens 123506041


### Build Word2Vec Models

In [48]:
vec_model_300 = Word2Vec(tokens,vector_size=300, min_count=1,workers=2)

In [49]:
vec_model_300.save("word2vec_300.model")
#vec_model_300 = Word2Vec.load("word2vec_300.model")

In [50]:
word_vectors_300 = vec_model_300.wv
word_vectors_300.save("word2vec_300.wordvectors")
#word_vectors_300 = KeyedVectors.load("word2vec_300.wordvectors",mmap="r")

In [ ]:
#http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/